# Spin-$s$ U(1)-eigenstate preparation

In [1]:
import numpy as np
from scipy.special import binom
import sys
import math
import itertools
np.set_printoptions(suppress=True)
np.set_printoptions(threshold=np.inf)

## Generating Gray code for bounded integer compositions (classical)

### Hamiltonian search

In [2]:
# Code generated using Gemini
def gray_Hamiltonian(n, k, d):
    # 1. Generate all valid configurations (lexicographically)
    states = []
    def generate_all(idx, current_sum, current_path):
        if idx == n - 1:
            val = k - current_sum
            if 0 <= val < d:
                states.append(tuple(current_path + [val]))
            return
        
        low = max(0, (k - current_sum) - (n - idx - 1) * (d - 1))
        high = min(d - 1, k - current_sum)
        for v in range(low, high + 1):
            generate_all(idx + 1, current_sum + v, current_path + [v])

    generate_all(0, 0, [])
    
    # 2. Pre-calculate adjacency (L1 distance == 2)
    # We use a dict for fast lookup of unvisited neighbors
    adj = {state: [] for state in states}
    for i in range(len(states)):
        for j in range(i + 1, len(states)):
            s1, s2 = states[i], states[j]
            # L1 distance check
            diff = sum(abs(a - b) for a, b in zip(s1, s2))
            if diff == 2:
                adj[s1].append(s2)
                adj[s2].append(s1)

    # 3. Find Hamiltonian Path using DFS + Warnsdorff's Heuristic
    # Heuristic: Always visit the neighbor with the fewest remaining neighbors
    path = []
    visited = set()

    def solve(curr):
        path.append(list(curr))
        visited.add(curr)
        
        if len(path) == len(states):
            return True
        
        # Sort neighbors by their current "degree" in the unvisited graph
        neighbors = [n for n in adj[curr] if n not in visited]
        neighbors.sort(key=lambda x: len([m for m in adj[x] if m not in visited]))
        
        for neighbor in neighbors:
            if solve(neighbor):
                return True
        
        # Backtrack
        visited.remove(curr)
        path.pop()
        return False

    # Start from the first lexicographical state
    if solve(states[0]):
        return path
    else:
        return "No Gray code found for these parameters."


In [3]:
n, k, d = 3, 3, 3
gray_Hamiltonian(n, k, d)

[[0, 1, 2], [0, 2, 1], [1, 2, 0], [2, 1, 0], [1, 1, 1], [1, 0, 2], [2, 0, 1]]

In [4]:
n, k, d = 8, 7, 4
res = gray_Hamiltonian(n, k, d)

if isinstance(res, str):
    print(res)
else:
    print(f"Generated {len(res)} configurations.")
    violations = 0
    for i in range(len(res) - 1):
        diff = sum(abs(a - b) for a, b in zip(res[i], res[i+1]))
        if diff != 2:
            print(f"Violation at index {i}: {res[i]} -> {res[i+1]} (diff={diff})")
            violations += 1
    if violations == 0:
        print("Success! Every step is a valid Gray move (L1=2).")

Generated 2472 configurations.
Success! Every step is a valid Gray move (L1=2).


### Walsh (alternative)

## Computing parameters and angles for Gray gates (classical)

In [5]:
def myparams(vecs):
    dim=len(vecs)
    #untouched
    myu=[0]*dim
    myu[0]=[index for index, item in enumerate(vecs[0]) if item !=0]
    controls=[0]*dim
    setij=[0]*dim
    for l in range(dim-1):
        diff=list(np.array(vecs[l+1])-np.array(vecs[l]))
        try:
            i=diff.index(1)
            j=diff.index(-1)
        except ValueError:
            raise ValueError(f"Invalid transition at step {l}: vecs must differ by exactly (+1, -1).")
        setij[l]=[i,j]
    
        zero_positions = [index for index, item in enumerate(diff) if item == 0]
        myc=[index for index in zero_positions if vecs[l+1][index]*vecs[l][index] != 0 ]
        if l>0:
            myu[l]= list(set(myu[l-1])-set((i,j)))
        controls[l]= list(set(myc)-set(myu[l])) 
    return setij, controls       

In [6]:
def angleslist(n,coeffs, is_complex=False):
    dim=len(coeffs)
    thetas=[0]*(dim-1)
    phis=[0]*dim  
    
    if is_complex:
        for l in range(dim-1):    
            if l>=0 and l<dim-2:
                ss=0
                for j in range(l+1,dim):
                    ss=ss+np.abs(coeffs[j])**2
                thetas[l]=np.arctan2(np.sqrt(ss), np.abs(coeffs[l]))        
            if l==dim-2:
                thetas[l]=np.arctan2(np.abs(coeffs[l+1]), np.abs(coeffs[l]))

        for l in range(dim-1):
            if l>=0 and l<dim-2:
                denom=coeffs[l]*np.tan(thetas[l])*np.cos(thetas[l+1])
                phis[l]=np.angle(coeffs[l+1]/denom)
            if l==dim-2:
                denom=coeffs[l]*np.tan(thetas[l])
                phis[l]=np.angle(coeffs[l+1]/denom)

    else:
        # Real coefficients    
        for l in range(dim-1): 
            if l>=0 and l<dim-2:
                ss=0
                for j in range(l+1,dim):
                    ss=ss+coeffs[j]**2
                thetas[l]=np.arctan2(np.sqrt(ss), coeffs[l])        
            if l==dim-2:
                thetas[l]=np.arctan2(coeffs[l+1], coeffs[l])
    
    return thetas, phis

### Amplitudes for AKLT ground state (PBC)

In [7]:
def aklt_coeff(vec):
    #Map from our basis |0>, |1>, |2> to the 
    #basis |1>, |0>, |-1> that is used in AKLT MPS
    
    config = [1 - x for x in vec]

    """
    Computes the ground state coefficient Tr(A_m1 * A_m2 * ... * A_mN)
    for a given spin-1 configuration.
    
    Args:
        config: List of integers from {+1, 0, -1}
        
    Returns:
        The scalar coefficient (float).
    """
    # Define the 2x2 MPS matrices for the AKLT state
    A_plus  = np.array([[0, 1], [0, 0]], dtype=float)
    A_minus = np.array([[0, 0], [-1, 0]], dtype=float)
    A_zero  = (1/np.sqrt(2)) * np.array([[-1, 0], [0, 1]], dtype=float)
    
    # Map configuration values to matrices
    mapping = {1: A_plus, -1: A_minus, 0: A_zero}
    
    result = np.eye(2)
    
    for spin in config:
        if spin not in mapping:
            raise ValueError("Spins must be +1, 0, or -1")
        result = np.dot(result, mapping[spin])
        
    return np.trace(result)

def coefflistAKLT(n,vecs):
    dim=len(vecs)
    cc=[0]*dim
    for l in range(dim):
        cc[l]=aklt_coeff(vecs[l])
    # vector must be normalized
    ss=0
    for j in range(dim):
        ss=ss+np.abs(cc[j])**2
    norm = np.sqrt(ss)    
    return cc/norm  

### Amplitudes for spin-$s$ Dicke states

In [8]:
def coefflistDicke(n,s,vecs):
    dim=len(vecs)
    cc=[0]*dim
    for l in range(dim):
        prod=1
        for j in range(n):
            prod=prod*binom(2*s,vecs[l][j])
        cc[l]=np.sqrt(prod/binom(2*s*n,k))    
    return cc 

### Amplitudes for spin-$s$ Bethe states (PBC)

In [9]:
def sfunc(kj,kl,s):
    return 1-(1/(2.0*s))*(1-np.exp(1.j*kj))*(1-np.exp(1.j*kl))/(np.exp(1.j*kj)-np.exp(1.j*kl))

# kBethe=[k_0,...,k_{M-1}] specified Bethe roots
def A(kBethe,s):
    z=1
    for l in range(len(kBethe)):
        for j in range(l):
            z=z*sfunc(kBethe[j],kBethe[l],s)
    return z

# myperms is a collection of permutations t of the numbers 0 to M-1
def myperms(M):
    return list(itertools.permutations(range(M)))

# permutes the bethe roots k into an order determined by permutation t
def kperm(kBethe,t):
    y=[]
    for i in t:
        y.append(kBethe[i])
    return y

# x=[x_0,...,x_{M-1}] with 1 <= x_0 <= x_1 ... <= x_{M-1} <= n
def f(x,kBethe,s):
    z=0
    for t in myperms(len(kBethe)):
        y=kperm(kBethe,t)
        v=0
        for i in range(len(kBethe)):
            v=v+y[i]*x[i]
        phase=np.exp(1.j*v)
        z=z+A(y,s)*phase
    return z

# converts vec to x=[x_0,...,x_{M-1}] 
def myx(v):
    n=len(v)
    x=[]
    for i in range(n):
        x=x+[i+1]*v[i]
    return x      

def mj(x,j):
    all_positions = []
    for index, value in enumerate(x):
        if value == j:
            all_positions.append(index)
    return len(all_positions)    


def myfactor(x,n,s):
    prod=1
    for j in range(1,n+1):
        prod=prod*np.sqrt(binom(2*s,mj(x,j)))
    return prod

def coefflistBethe(n,kBethe,s,vecs):
    dim=len(vecs)
    cc=[0]*dim
    for l in range(dim):
        v=vecs[l]
        x=myx(v)
        cc[l]=f(x,kBethe,s)*myfactor(x,n,s)
        # "first" element must be real
        cc[l]=cc[l]*np.abs(cc[0])/cc[0]
    # vector must be normalized
    ss=0
    for j in range(dim):
        ss=ss+np.abs(cc[j])**2
    norm = np.sqrt(ss)    
    return cc/norm  

### Random real amplitudes

In [10]:
import random

random.seed(42)

def coefflistRandomReal(vecs):
    dim=len(vecs)
    random_reals = [random.random() - 0.5 for _ in range(dim)] 

    ss=0
    for j in range(dim):
        ss=ss+random_reals[j]**2
    norm = np.sqrt(ss) 
        
    return [x/norm for x in random_reals]

In [11]:
n, k, s = 3, 3, 1
d=int(2*s+1)
vecs=gray_Hamiltonian(n,k,d)
coefflistRandomReal(vecs)

[0.1780477022627338,
 -0.6065601775806494,
 -0.2872870451296978,
 -0.35345925337650674,
 0.3019731988316379,
 0.22564484069506555,
 0.500812409641123]

### Random complex amplitudes

In [12]:
def coefflistRandomComplex(vecs):
    dim=len(vecs)
    random_complex = [random.random() - 0.5 + 1j*random.random() for _ in range(dim)] 

    #make sure "first" element is real!
    random_complex[0] = np.real(random_complex[0])

    ss=0
    for j in range(dim):
        ss=ss+np.abs(random_complex[j])**2
    norm = np.sqrt(ss) 
        
    return [x/norm for x in random_complex]

In [13]:
n, k, s = 3, 3, 1
d=int(2*s+1)
vecs=gray_Hamiltonian(n,k,d)
coefflistRandomComplex(vecs)

[-0.2581661491879361,
 (-0.2938800613184398+0.13665027961985807j),
 (0.003347092916620397+0.01658516861282361j),
 (-0.18822859698371391+0.40618236709765976j),
 (0.02808874303046633+0.1377769468844357j),
 (0.055791683366341466+0.5058997566054636j),
 (-0.30844176334562495+0.5036427280527629j)]

## Hamiltonians for checking eigenstates

In [14]:
from scipy import sparse

def spinmats(s):
    d = int(2*s + 1)
    
    # m values from s down to -s
    m_values = np.linspace(s, -s, d)
    
    Sz = sparse.diags([m_values], [0], shape=(d,d), format='csr')
    
    off_diag = np.sqrt(s*(s+1) - m_values[:-1]*(m_values[:-1]-1))
    Sp = sparse.diags([off_diag], [1], shape=(d,d), format='csr')
    Sm = sparse.diags([off_diag], [-1], shape=(d,d), format='csr')
            
    return Sp, Sm, Sz

### AKLT Hamiltonian (PBC)

In [15]:
def HamAKLT(n):
    #spin-1 
    s=1
    d = int(2*s + 1)
    Sp, Sm, Sz = spinmats(s)
    I_site = sparse.eye(d, format='csr')

    def get_full_space_op(op, site_idx):
        """Embeds a single-site operator into the n-site Hilbert space."""
        # op_1 ⊗ op_2 ⊗ ... ⊗ op_n
        op_list = [I_site] * n
        op_list[site_idx] = op
        
        full_op = op_list[0]
        for i in range(1, n):
            full_op = sparse.kron(full_op, op_list[i], format='csr')
        return full_op  

    H = sparse.csr_matrix((d**n, d**n))

    id = sparse.eye(d**n, format='csr')

    for i in range(n):
        j = (i + 1) % n

        term_z = get_full_space_op(Sz, i) @ get_full_space_op(Sz, j)
        term_pm = 0.5 * (get_full_space_op(Sp, i) @ get_full_space_op(Sm, j) + 
                         get_full_space_op(Sm, i) @ get_full_space_op(Sp, j))

        S_dot_S = term_z + term_pm

        H += 0.5*S_dot_S + (1.0/6.0)*S_dot_S@S_dot_S+ (1.0/3.0)*id
    return H

### spin-$s$ Dicke Hamiltonian

In [16]:
def HamDicke(s,n):
    d = int(2*s + 1)
    Sp, Sm, Sz = spinmats(s)
    I_site = sparse.eye(d, format='csr')

    def get_full_space_op(op, site_idx):
        """Embeds a single-site operator into the n-site Hilbert space."""
        # op_1 ⊗ op_2 ⊗ ... ⊗ op_n
        op_list = [I_site] * n
        op_list[site_idx] = op
        
        full_op = op_list[0]
        for i in range(1, n):
            full_op = sparse.kron(full_op, op_list[i], format='csr')
        return full_op  

    SS_z = sparse.csr_matrix((d**n, d**n))
    SS_p = SS_z
    SS_m = SS_z

    for i in range(n):       
        SS_z += get_full_space_op(Sz, i) 
        SS_p += get_full_space_op(Sp, i) 
        SS_m += get_full_space_op(Sm, i) 

    SS = SS_z @ SS_z + 0.5 * (SS_p @ SS_m + SS_m @ SS_p)
    
    return -SS

### integrable spin-$s$ XXX Hamiltonian (PBC) 

In [17]:
def HamXXX(s,n):
    #works only for s=1/2, 1, 3/2
    d = int(2*s + 1)
    Sp, Sm, Sz = spinmats(s)
    I_site = sparse.eye(d, format='csr')

    def get_full_space_op(op, site_idx):
        """Embeds a single-site operator into the n-site Hilbert space."""
        # op_1 ⊗ op_2 ⊗ ... ⊗ op_n
        op_list = [I_site] * n
        op_list[site_idx] = op
        
        full_op = op_list[0]
        for i in range(1, n):
            full_op = sparse.kron(full_op, op_list[i], format='csr')
        return full_op  

    H = sparse.csr_matrix((d**n, d**n))

    id = sparse.eye(d**n, format='csr')

    for i in range(n):
        j = (i + 1) % n

        term_z = get_full_space_op(Sz, i) @ get_full_space_op(Sz, j)
        term_pm = 0.5 * (get_full_space_op(Sp, i) @ get_full_space_op(Sm, j) + 
                         get_full_space_op(Sm, i) @ get_full_space_op(Sp, j))

        S_dot_S = term_z + term_pm

        if s==1/2:
            H += 2.0*S_dot_S - 0.5*id

        elif s==1:
            H += 0.5 * S_dot_S - 0.5 *S_dot_S@S_dot_S
            
        elif s==3/2:
            H += (-(1.0/8.0)*S_dot_S + (1.0/27.0)*S_dot_S@S_dot_S +
            (2.0/27.0)*S_dot_S@S_dot_S@S_dot_S - (0.75)*id )
        
    return H

def energy(kBethe, s):
    m=len(kBethe)
    ee=0
    for i in range(m):
        ee = ee - (2.0/s)*np.sin(kBethe[i]/2)**2
    return ee    

## Quantum gates

In [18]:
# pip install cirq

In [19]:
import cirq

simulator=cirq.Simulator()

In [20]:
class oplus(cirq.Gate):
    def __init__(self, d, power):
        super(oplus, self)
        self.d=d
        self.power = power
        
    def _qid_shape_(self):
        return (self.d,)
        
    def _unitary_(self):
        v=np.ones((self.d-1,), dtype=int)
        mat=np.diag(v,-1)
        mat[0,self.d-1]=1
        matp=np.linalg.matrix_power(mat, self.power)
        return matp
        
    def _circuit_diagram_info_(self, args):
        return f"oplus {self.power}"

class ominus(cirq.Gate):
    def __init__(self, d, power):
        super(ominus, self)
        self.d=d
        self.power = power
        
    def _qid_shape_(self):
        return (self.d,)
        
    def _unitary_(self):
        v=np.ones((self.d-1,), dtype=int)
        mat=np.diag(v,1)
        mat[self.d-1,0]=1
        matp=np.linalg.matrix_power(mat, self.power)
        return matp
        
    def _circuit_diagram_info_(self, args):
        return f"ominus {self.power}"

class R(cirq.Gate):
    def __init__(self, d, theta, phi, i, j):
        if i == j or not (0 <= i < d) or not (0 <= j < d):
            raise ValueError("Indices i and j must be different and in range [0, d-1]")
        super().__init__()
        self.d = d
        self.theta = theta
        self.phi = phi
        self.i = i
        self.j = j
        
    def _qid_shape_(self):
        return (self.d,)

    def _unitary_(self):
        U = np.eye(self.d, dtype=np.complex128)
        c, s, phase = np.cos(self.theta), np.sin(self.theta), np.exp(self.phi*1j)
        U[self.i, self.i] = c
        U[self.j, self.j] = phase*c
        U[self.i, self.j] = -s
        U[self.j, self.i] = phase*s
        return U

    def _circuit_diagram_info_(self, args):
        return f"R{self.i}{self.j}({self.theta:.2f})"


In [21]:
def GrayGate(qr,n,k,s,vecs,setij,controls,thetas,phis,l):
    """Gives generator"""
    d=int(2*s+1)
    i=setij[l][0]
    j=setij[l][1]

    zero_positions = controls[l]
  
    numzeros = len(zero_positions)

    mytheta=thetas[l]

    myphi=phis[l]

    extra_control_qudits = [qr[m] for m in zero_positions]

    extra_control_values = [vecs[l][m] for m in zero_positions]

    coplus_vals = extra_control_values + [vecs[l][j]]
    
    total_controls = len(coplus_vals)

    Coplus = cirq.ControlledGate(oplus(d, 1), 
        num_controls=total_controls,
        control_values=tuple(coplus_vals),
        control_qid_shape=tuple([d] * total_controls))

    cominus_vals = extra_control_values + [vecs[l][j]]

    Cominus = cirq.ControlledGate(ominus(d, 1), 
        num_controls=total_controls, 
        control_values=tuple(cominus_vals),
        control_qid_shape=tuple([d] * total_controls))

    cr_vals = extra_control_values + [vecs[l][i] + 1]

    CR = cirq.ControlledGate(R(d, mytheta, myphi, vecs[l][j], vecs[l][j]-1), 
        num_controls=total_controls, 
        control_values=tuple(cr_vals),
        control_qid_shape=tuple([d] * total_controls))


    yield Coplus(*extra_control_qudits, qr[j], qr[i])
    yield CR(*extra_control_qudits, qr[i], qr[j])
    yield Cominus(*extra_control_qudits, qr[j], qr[i])
    

Initial state:

In [22]:
def initstate(qr,n,k,s,vecs):
    """Gives generator"""
    # initial state
    d=int(2*s+1)
    init=vecs[0]
    for j in range(n):
        yield oplus(d,init[j])(qr[j])   

## Quantum circuits

Generate states and check that they are eigenstates of the corresponding Hamiltonians

### AKLT ground state

In [23]:
def myAKLTstate(qr,n):
    """Gives generator"""
    k=n
    s=1
    d=int(2*s+1)
    vecs=gray_Hamiltonian(n,k,d)
    setij, controls=myparams(vecs)
    coeffs=coefflistAKLT(n,vecs)
    thetas, phis=angleslist(n,coeffs,False)
    # initial state
    yield initstate(qr,n,k,s,vecs)
    # apply Gray gates
    for l in range(len(vecs)-1):
        yield GrayGate(qr,n,k,s,vecs,setij,controls,thetas,phis,l)

In [24]:
n=4
s=1
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myAKLTstate(qr,n))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

-0.22|0112⟩ + 0.22|0121⟩ + 0.44|0202⟩ - 0.22|0211⟩ + 0.22|1012⟩ - 0.22|1021⟩ - 0.22|1102⟩ + 0.22|1111⟩ - 0.22|1120⟩ - 0.22|1201⟩ + 0.22|1210⟩ - 0.22|2011⟩ + 0.44|2020⟩ + 0.22|2101⟩ - 0.22|2110⟩


In [25]:
psi=result.final_state_vector
mydif=HamAKLT(n) @ psi 
print(mydif.real.round(6))

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0. -0.  0.
  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. -0.  0. -0.  0. -0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.
  0.  0. -0.  0. -0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.
 -0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [26]:
n=6
s=1
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myAKLTstate(qr,n))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

-0.07|011112⟩ + 0.07|011121⟩ + 0.15|011202⟩ - 0.07|011211⟩ + 0.15|012012⟩ - 0.15|012021⟩ - 0.15|012102⟩ + 0.07|012111⟩ + 0.15|020112⟩ - 0.15|020121⟩ - 0.3|020202⟩ + 0.15|020211⟩ - 0.15|021012⟩ + 0.15|021021⟩ + 0.15|021102⟩ - 0.07|021111⟩ + 0.07|101112⟩ - 0.07|101121⟩ - 0.15|101202⟩ + 0.07|101211⟩ - 0.15|102012⟩ + 0.15|102021⟩ + 0.15|102102⟩ - 0.07|102111⟩ - 0.07|110112⟩ + 0.07|110121⟩ + 0.15|110202⟩ - 0.07|110211⟩ + 0.07|111012⟩ - 0.07|111021⟩ - 0.07|111102⟩ + 0.07|111111⟩ - 0.07|111120⟩ - 0.07|111201⟩ + 0.07|111210⟩ - 0.07|112011⟩ + 0.15|112020⟩ + 0.07|112101⟩ - 0.07|112110⟩ - 0.07|120111⟩ + 0.15|120120⟩ + 0.15|120201⟩ - 0.15|120210⟩ + 0.07|121011⟩ - 0.15|121020⟩ - 0.07|121101⟩ + 0.07|121110⟩ - 0.07|201111⟩ + 0.15|201120⟩ + 0.15|201201⟩ - 0.15|201210⟩ + 0.15|202011⟩ - 0.3|202020⟩ - 0.15|202101⟩ + 0.15|202110⟩ + 0.07|210111⟩ - 0.15|210120⟩ - 0.15|210201⟩ + 0.15|210210⟩ - 0.07|211011⟩ + 0.15|211020⟩ + 0.07|211101⟩ - 0.07|211110⟩


In [27]:
psi=result.final_state_vector
mydif=HamAKLT(n) @ psi 
print(mydif.real.round(6))

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.
  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.
  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0. -0.  0.
  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. -0.  0. -0.  0. -0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.
  0.  0. -0.  0.  0.  0.  0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

### Spin-s Dicke states

In [28]:
def myDickestate(qr,n,k,s):
    """Gives generator"""
    d=int(2*s+1)
    vecs=gray_Hamiltonian(n,k,d)
    setij, controls=myparams(vecs)
    coeffs=coefflistDicke(n,s,vecs)
    thetas, phis=angleslist(n,coeffs,False)
    # initial state
    yield initstate(qr,n,k,s,vecs)
    # apply Gray gates
    for l in range(len(vecs)-1):
        yield GrayGate(qr,n,k,s,vecs,setij,controls,thetas,phis,l)

#### $s=1/2$

In [29]:
n, k, s = 4, 2, 1/2
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myDickestate(qr,n,k,s))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.41|0011⟩ + 0.41|0101⟩ + 0.41|0110⟩ + 0.41|1001⟩ + 0.41|1010⟩ + 0.41|1100⟩


In [30]:
psi=result.final_state_vector
mydif=HamDicke(s,n) @ psi + s*n*(s*n+1) * psi
print(mydif.real.round(6))

[ 0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0.  0. -0.  0.  0.  0.]


In [31]:
n, k, s = 6, 4, 1/2
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myDickestate(qr,n,k,s))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.26|001111⟩ + 0.26|010111⟩ + 0.26|011011⟩ + 0.26|011101⟩ + 0.26|011110⟩ + 0.26|100111⟩ + 0.26|101011⟩ + 0.26|101101⟩ + 0.26|101110⟩ + 0.26|110011⟩ + 0.26|110101⟩ + 0.26|110110⟩ + 0.26|111001⟩ + 0.26|111010⟩ + 0.26|111100⟩


In [32]:
psi=result.final_state_vector
mydif=HamDicke(s,n) @ psi + s*n*(s*n+1) * psi
print(mydif.real.round(6))

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.]


#### $s=1$

In [33]:
n, k, s = 3, 3, 1
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myDickestate(qr,n,k,s))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.32|012⟩ + 0.32|021⟩ + 0.32|102⟩ + 0.63|111⟩ + 0.32|120⟩ + 0.32|201⟩ + 0.32|210⟩


In [34]:
psi=result.final_state_vector
mydif=HamDicke(s,n) @ psi + s*n*(s*n+1) * psi
print(mydif.real.round(6))

[ 0.        0.        0.        0.        0.       -0.        0.
 -0.        0.        0.        0.       -0.        0.        0.000001
  0.       -0.        0.        0.        0.       -0.        0.
 -0.        0.        0.        0.        0.        0.      ]


In [35]:
n, k, s = 3, 4, 1
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myDickestate(qr,n,k,s))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.26|022⟩ + 0.52|112⟩ + 0.52|121⟩ + 0.26|202⟩ + 0.52|211⟩ + 0.26|220⟩


In [36]:
psi=result.final_state_vector
mydif=HamDicke(s,n) @ psi + s*n*(s*n+1) * psi
print(mydif.real.round(6))

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.
  0.  0. -0.  0.  0.  0. -0.  0.  0.]


#### $s=3/2$

In [37]:
n, k, s = 4, 4, 3/2
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myDickestate(qr,n,k,s))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.08|0013⟩ + 0.13|0022⟩ + 0.08|0031⟩ + 0.08|0103⟩ + 0.23|0112⟩ + 0.23|0121⟩ + 0.08|0130⟩ + 0.13|0202⟩ + 0.23|0211⟩ + 0.13|0220⟩ + 0.08|0301⟩ + 0.08|0310⟩ + 0.08|1003⟩ + 0.23|1012⟩ + 0.23|1021⟩ + 0.08|1030⟩ + 0.23|1102⟩ + 0.4|1111⟩ + 0.23|1120⟩ + 0.23|1201⟩ + 0.23|1210⟩ + 0.08|1300⟩ + 0.13|2002⟩ + 0.23|2011⟩ + 0.13|2020⟩ + 0.23|2101⟩ + 0.23|2110⟩ + 0.13|2200⟩ + 0.08|3001⟩ + 0.08|3010⟩ + 0.08|3100⟩


In [38]:
psi=result.final_state_vector
mydif=HamDicke(s,n) @ psi + s*n*(s*n+1) * psi
print(mydif.real.round(6))

[ 0.        0.        0.        0.        0.        0.        0.
 -0.        0.        0.       -0.        0.        0.       -0.
  0.        0.        0.        0.        0.       -0.        0.
  0.        0.000001  0.        0.       -0.000001  0.        0.
  0.        0.        0.        0.        0.        0.       -0.
  0.        0.       -0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
 -0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.        0.        0.        0.
  0.        0.        0.        0.       -0.        0.        0.
  0.000001  0.        0.        0.        0.        0.       -0.
  0.        0.        0.        0.        0.        0.000001  0.
  0.        0.000001  0.        0.       -0.        0.        0.
  0.        0.        0.        0.        0.        0.       -0.
  0.        0.       -0.000001  0.        0.        0.        0.
  0.        0.        0. 

### Spin-$s$ Bethe states

In [39]:
def myBethestate(qr,n,k,s,kBethe):
    """Gives generator"""
    d=int(2*s+1)
    vecs=gray_Hamiltonian(n,k,d)
    setij, controls=myparams(vecs)
    coeffs=coefflistBethe(n,kBethe,s,vecs)
    thetas, phis=angleslist(n,coeffs,True)
    # initial state
    yield initstate(qr,n,k,s,vecs)
    # apply Gray gates
    for l in range(len(vecs)-1):
        yield GrayGate(qr,n,k,s,vecs,setij,controls,thetas,phis,l)

#### $s=1/2$

In [40]:
n=4
k=2
s=1/2
kBethe=[-2.0943951115532506, 2.0943951115532506]
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myBethestate(qr,n,k,s,kBethe))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.29|0011⟩ - 0.58|0101⟩ + 0.29|0110⟩ + 0.29|1001⟩ - 0.58|1010⟩ + 0.29|1100⟩


In [41]:
psi=result.final_state_vector
mydif=HamXXX(s,n) @ psi - energy(kBethe, s) * psi
print(mydif.real.round(6))
print(f"Energy= {energy(kBethe, s)}")

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Energy= -6.000000031731362


In [42]:
n=8
k=4
s=1/2
kBethe=[1.522002484283222, 2.6348317661215677, -1.522002484283222, -2.6348317661215677]
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myBethestate(qr,n,k,s,kBethe))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.01|00001111⟩ - 0.03|00010111⟩ + 0.05|00011011⟩ - 0.03|00011101⟩ + 0.01|00011110⟩ + 0.05|00100111⟩ - 0.16|00101011⟩ + 0.14|00101101⟩ - 0.03|00101110⟩ + 0.09|00110011⟩ - 0.16|00110101⟩ + 0.05|00110110⟩ + 0.05|00111001⟩ - 0.03|00111010⟩ + 0.01|00111100⟩ - 0.03|01000111⟩ + 0.14|01001011⟩ - 0.16|01001101⟩ + 0.05|01001110⟩ - 0.16|01010011⟩ + 0.4|01010101⟩ - 0.16|01010110⟩ - 0.16|01011001⟩ + 0.14|01011010⟩ - 0.03|01011100⟩ + 0.05|01100011⟩ - 0.16|01100101⟩ + 0.09|01100110⟩ + 0.14|01101001⟩ - 0.16|01101010⟩ + 0.05|01101100⟩ - 0.03|01110001⟩ + 0.05|01110010⟩ - 0.03|01110100⟩ + 0.01|01111000⟩ + 0.01|10000111⟩ - 0.03|10001011⟩ + 0.05|10001101⟩ - 0.03|10001110⟩ + 0.05|10010011⟩ - 0.16|10010101⟩ + 0.14|10010110⟩ + 0.09|10011001⟩ - 0.16|10011010⟩ + 0.05|10011100⟩ - 0.03|10100011⟩ + 0.14|10100101⟩ - 0.16|10100110⟩ - 0.16|10101001⟩ + 0.4|10101010⟩ - 0.16|10101100⟩ + 0.05|10110001⟩ - 0.16|10110010⟩ + 0.14|10110100⟩ - 0.03|10111000⟩ + 0.01|11000011⟩ - 0.03|11000101⟩ + 0.05|11000110⟩ + 0.05|11001001⟩ -

In [43]:
psi=result.final_state_vector
mydif=HamXXX(s,n) @ psi - energy(kBethe, s) * psi
print(mydif.real.round(6))
print(f"Energy= {energy(kBethe, s)}")

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0. -0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0. -0.
 -0.  0.  0. -0. -0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.
  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0. -0.  0.  0. -0.
  0.  0. -0.  0.  0.  0.  0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0. -0. -0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0.
  0.  0.  0. -0.  0.  0. -0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0.
  0. -0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 -0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.
 -0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0

#### $s=1$

In [44]:
n=4
k=2
s=1
kBethe=[2.356194490192345 + 0.8813735870195432*1j,2.356194490192345 - 0.8813735870195432*1j]
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myBethestate(qr,n,k,s,kBethe))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.41|0002⟩ + (-0.2-0.2j)|0011⟩ + 0.41j|0020⟩ + (0.2-0.2j)|0110⟩ - 0.41|0200⟩ + (-0.2+0.2j)|1001⟩ + (0.2+0.2j)|1100⟩ - 0.41j|2000⟩


In [45]:
psi=result.final_state_vector
mydif=HamXXX(s,n) @ psi - energy(kBethe, s) * psi
print(mydif.real.round(7))
print(f"Energy= {energy(kBethe, s)}")

[ 0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.
 -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Energy= (-4+0j)


In [46]:
n=4
k=3
s=1
kBethe=[-3.141592653589793,3.141592653589793 - 0.962423650119207*1j,-3.141592653589793 + 0.962423650119207*1j]
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myBethestate(qr,n,k,s,kBethe))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.16|0012⟩ - 0.16|0021⟩ - 0.32|0102⟩ + 0.32|0111⟩ - 0.16|0120⟩ - 0.32|0201⟩ + 0.16|0210⟩ + 0.16|1002⟩ - 0.32|1011⟩ + 0.32|1020⟩ + 0.32|1101⟩ - 0.32|1110⟩ + 0.16|1200⟩ - 0.16|2001⟩ + 0.32|2010⟩ - 0.16|2100⟩


In [47]:
psi=result.final_state_vector
mydif=HamXXX(s,n) @ psi - energy(kBethe, s) * psi
print(mydif.real.round(6))
print(f"Energy= {energy(kBethe, s)}")

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0. -0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0. -0.  0.  0.
  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. -0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Energy= (-7+2.738393491321014e-16j)


#### $s=3/2$

In [48]:
n=4
k=2
s=3/2
kBethe=[2.356194490192345 - 0.3465735902799727*1j,2.356194490192345 + 0.3465735902799727*1j]
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myBethestate(qr,n,k,s,kBethe))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.39|0002⟩ + (-0.22-0.22j)|0011⟩ + 0.39j|0020⟩ + (0.22-0.22j)|0110⟩ - 0.39|0200⟩ + (-0.22+0.22j)|1001⟩ + (0.22+0.22j)|1100⟩ - 0.39j|2000⟩


In [49]:
psi=result.final_state_vector
mydif=HamXXX(s,n) @ psi - energy(kBethe, s) * psi
print(mydif.real.round(6))
print(f"Energy= {energy(kBethe, s)}")

[ 0.  0.  0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

In [50]:
n=4
k=3
s=3/2
kBethe=[1.4650556470460385 + 0.8801868182958004*1j,1.7822776862926129,1.4650556470460385 - 0.8801868182958004*1j]
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myBethestate(qr,n,k,s,kBethe))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

0.16|0003⟩ + (0.01-0.24j)|0012⟩ + (-0.24+0.01j)|0021⟩ + 0.16j|0030⟩ - 0.18|0102⟩ + 0.27j|0111⟩ + (0.24+0.01j)|0120⟩ + 0.18|0201⟩ + (-0.01-0.24j)|0210⟩ - 0.16|0300⟩ + (0.01+0.24j)|1002⟩ + 0.27|1011⟩ - 0.18j|1020⟩ - 0.27j|1101⟩ - 0.27|1110⟩ + (-0.01+0.24j)|1200⟩ + (-0.24-0.01j)|2001⟩ + 0.18j|2010⟩ + (0.24-0.01j)|2100⟩ - 0.16j|3000⟩


In [51]:
psi=result.final_state_vector
mydif=HamXXX(s,n) @ psi - energy(kBethe, s) * psi
print(mydif.real.round(6))
print(f"Energy= {energy(kBethe, s)}")

[ 0.  0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0. -0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0. -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0

### State with random real amplitudes

In [52]:
def myRandomRealstate(qr,n,k,s):
    """Gives generator"""
    d=int(2*s+1)
    vecs=gray_Hamiltonian(n,k,d)
    setij, controls=myparams(vecs)
    coeffs=coefflistRandomReal(vecs)
    thetas, phis=angleslist(n,coeffs,False)
    print([[vecs[l], coeffs[l]] for l in range(len(vecs))])
    # initial state
    yield initstate(qr,n,k,s,vecs)
    # apply Gray gates
    for l in range(len(vecs)-1):
        yield GrayGate(qr,n,k,s,vecs,setij,controls,thetas,phis,l)

In [53]:
n, k, s = 3, 3, 1
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myRandomRealstate(qr,n,k,s))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

[[[0, 1, 2], 0.22914141071320815], [[0, 2, 1], -0.1847447954908145], [[1, 2, 0], -0.3984261354862782], [[2, 1, 0], 0.5287512277313999], [[1, 1, 1], -0.18897280095839092], [[1, 0, 2], -0.4709754563055751], [[2, 0, 1], -0.4663836706244343]]
0.23|012⟩ - 0.18|021⟩ - 0.47|102⟩ - 0.19|111⟩ - 0.4|120⟩ - 0.47|201⟩ + 0.53|210⟩


### State with random complex amplitudes

In [54]:
def myRandomComplexstate(qr,n,k,s):
    """Gives generator"""
    d=int(2*s+1)
    vecs=gray_Hamiltonian(n,k,d)
    setij, controls=myparams(vecs)
    coeffs=coefflistRandomComplex(vecs)
    thetas, phis=angleslist(n,coeffs,True)
    print([[vecs[l], coeffs[l]] for l in range(len(vecs))])
    # initial state
    yield initstate(qr,n,k,s,vecs)
    # apply Gray gates
    for l in range(len(vecs)-1):
        yield GrayGate(qr,n,k,s,vecs,setij,controls,thetas,phis,l)

In [55]:
n, k, s = 3, 3, 1
d=int(2*s+1)
qr=cirq.LineQid.range(n, dimension=d)
test=cirq.Circuit(myRandomComplexstate(qr,n,k,s))
#print(test)
result=simulator.simulate(test)
print(cirq.dirac_notation(result.final_state_vector, qid_shape=(d,)*n))

[[[0, 1, 2], 0.2001412397582974], [[0, 2, 1], (0.17689217245178787+0.42029292737915425j)], [[1, 2, 0], (0.020865763131741745+0.5604712314571245j)], [[2, 1, 0], (-0.06995877541573142+0.3179507555800248j)], [[1, 1, 1], (0.18972237903805095+0.3562397611056414j)], [[1, 0, 2], (0.20832702474648057+0.33252905750207323j)], [[2, 0, 1], (0.11782424374255024+0.02639279897514557j)]]
0.2|012⟩ + (0.18+0.42j)|021⟩ + (0.21+0.33j)|102⟩ + (0.19+0.36j)|111⟩ + (0.02+0.56j)|120⟩ + (0.12+0.03j)|201⟩ + (-0.07+0.32j)|210⟩
